# ⚡ MAI-UI on B200: Next-Gen Blackwell Performance

This notebook runs MAI-UI on NVIDIA B200 GPU (Blackwell architecture).

**B200 Advantages:**
- 192GB HBM3e (12x more VRAM than T4)
- 8.0 TB/s bandwidth (25x faster than T4)
- FP4 Tensor Cores (4x throughput vs FP16)
- 5th Gen Tensor Cores
- 2nd Gen Transformer Engine

**Note:** B200 features are bleeding-edge. Some vLLM optimizations may require updates.


In [ ]:
# Cell 1: Check GPU and Install Dependencies
import subprocess, sys

print("=" * 70)
print("🔍 GPU DETECTION")
print("=" * 70)
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    cc = torch.cuda.get_device_capability()
    
    print(f"\n✅ GPU: {gpu_name}")
    print(f"✅ Memory: {gpu_memory:.1f} GB")
    print(f"✅ Compute Capability: SM {cc[0]}.{cc[1]}")
    
    if "B200" in gpu_name or "B100" in gpu_name or cc[0] >= 10:
        print("\n⚡ Blackwell detected - using next-gen settings!")
    elif cc[0] >= 9:
        print("\n🚀 Hopper detected - using H100 settings (close to B200)")
    else:
        print(f"\n⚠️ Expected B200, got {gpu_name}")
else:
    print("❌ No GPU!")
    sys.exit(1)

print("\n📦 Installing dependencies...")
%pip install -q vllm>=0.6.0 pillow requests jinja2
print("✅ Dependencies installed!")


In [ ]:
# Cell 2: B200-Optimized Configuration
from vllm import LLM, SamplingParams
import time

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │                        B200 MEMORY BUDGET (192 GB)                          │
# ├─────────────────────────────────────────────────────────────────────────────┤
# │  Component             │  Size      │  Notes                               │
# ├─────────────────────────────────────────────────────────────────────────────┤
# │  Model Weights (BF16)  │  ~16 GB    │  8B params × 2 bytes                 │
# │  KV Cache              │  ~100 GB   │  128K context × 128 seqs             │
# │  Activations           │  ~30 GB    │  Massive batch support               │
# │  CUDA Graphs           │  ~5 GB     │  Pre-compiled kernels                │
# │  Prefix Cache          │  ~30 GB    │  Reusable KV blocks                  │
# │  Headroom              │  ~11 GB    │  Safety margin                       │
# └─────────────────────────────────────────────────────────────────────────────┘

B200_CONFIG = {
    "model": "Tongyi-MAI/MAI-UI-8B",  # Can run 8B at full precision!
    "trust_remote_code": True,
    "dtype": "bfloat16",
    "gpu_memory_utilization": 0.95,
    "max_model_len": 65536,  # 64K context (vs 2K on T4)
    "max_num_seqs": 128,  # 128 concurrent (vs 4 on T4)
    "enforce_eager": False,  # CUDA Graphs enabled
    "limit_mm_per_prompt": {"image": 16, "video": 4},  # Multiple 4K images
    "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 4147200},  # 4K resolution
    "enable_prefix_caching": True,
    "enable_chunked_prefill": True,
}

print("⚡ B200-OPTIMIZED CONFIGURATION")
print("=" * 70)
for k, v in B200_CONFIG.items():
    print(f"  {k}: {v}")
print("=" * 70)


In [ ]:
# Cell 3: Initialize vLLM Engine
print("⚡ Initializing vLLM with B200 optimizations...")
print("   - FlashAttention (latest version)")
print("   - FP4 quantization (when available)")
print("   - 64K context, 128 concurrent sequences\n")

init_start = time.time()
llm = LLM(**B200_CONFIG)
init_time = time.time() - init_start

print(f"\n✅ Engine initialized in {init_time:.1f}s")

allocated = torch.cuda.memory_allocated() / (1024**3)
reserved = torch.cuda.memory_reserved() / (1024**3)
print(f"\n📊 GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")
print(f"📊 Available: {192 - reserved:.1f} GB free")


In [ ]:
# Cell 4: MAI-UI Prompt Format
import re, json

MAI_PROMPT = """You are a GUI grounding agent. Locate the UI element described.
Output: <grounding_think>[reasoning]</grounding_think><answer>{"coordinate": [x, y]}</answer>"""

def build_prompt(instruction):
    return (f"<|im_start|>system\n{MAI_PROMPT}<|im_end|>\n"
            f"<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>{instruction}<|im_end|>\n"
            "<|im_start|>assistant\n")

def parse_response(text):
    result = {"thinking": None, "coordinate": None}
    think = re.search(r"<grounding_think>(.*?)</grounding_think>", text, re.DOTALL)
    if think: result["thinking"] = think.group(1).strip()
    answer = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL)
    if answer:
        try:
            data = json.loads(answer.group(1).strip())
            if "coordinate" in data:
                result["coordinate"] = [data["coordinate"][0]/999, data["coordinate"][1]/999]
        except: pass
    return result

print("✅ Prompt functions defined")


In [ ]:
# Cell 5: Create 4K Test Image
from PIL import Image, ImageDraw

def create_4k_screenshot():
    """Create 4K test screenshot (3840x2160)."""
    img = Image.new('RGB', (3840, 2160), '#0a0a1a')
    draw = ImageDraw.Draw(img)
    
    # Top bar
    draw.rectangle([0, 0, 3840, 100], fill='#1a1a3a')
    draw.text((40, 35), "Enterprise Dashboard - 4K Resolution", fill='#eee')
    
    # Sidebar
    draw.rectangle([0, 100, 400, 2160], fill='#0f1f3f')
    for i, item in enumerate(["Dashboard", "Analytics", "Reports", "Users", "Settings", "Admin", "Logs"]):
        draw.text((50, 150 + i*100), f"📊 {item}", fill='#c0c0c0')
    
    # Main content - large cards
    for row in range(2):
        for col in range(4):
            x, y = 450 + col*800, 150 + row*500
            draw.rectangle([x, y, x+750, y+450], fill='#1f2f4f', outline='#3a5a8a')
            draw.text((x+30, y+30), f"Metric {row*4 + col + 1}", fill='#888')
            draw.text((x+30, y+100), f"${(row*4 + col + 1) * 12345:,}", fill='#00ff88')
    
    # Action buttons
    for i, (text, color) in enumerate([("Export All", '#4a90d9'), ("Generate Report", '#e94560'),
                                        ("Sync Data", '#0f9b58'), ("Configure", '#ff9800')]):
        x = 450 + i*400
        draw.rectangle([x, 1150, x+350, 1220], fill=color)
        draw.text((x+50, 1170), text, fill='white')
    
    return img

test_image = create_4k_screenshot()
print(f"📸 Created 4K image: {test_image.size}")
thumb = test_image.copy(); thumb.thumbnail((800, 450)); display(thumb)


In [ ]:
# Cell 6: Run Inference
sampling_params = SamplingParams(temperature=0.0, max_tokens=512, stop=["<|im_end|>"])

instructions = ["Click on Analytics", "Click on Export All", "Click on Metric 1", 
                "Click on Settings", "Click on Generate Report", "Click on Admin"]

print("⚡ Running B200-optimized inference...")
results = []
for i, inst in enumerate(instructions, 1):
    inputs = {"prompt": build_prompt(inst), "multi_modal_data": {"image": test_image}}
    start = time.time()
    outputs = llm.generate([inputs], sampling_params=sampling_params)
    latency = (time.time() - start) * 1000
    parsed = parse_response(outputs[0].outputs[0].text)
    results.append({"instruction": inst, "latency_ms": latency, "parsed": parsed})
    coord = parsed.get("coordinate", "N/A")
    print(f"[{i}] {inst:30} → {str(coord):20} {latency:.0f}ms")

avg = sum(r["latency_ms"] for r in results) / len(results)
print(f"\n📊 Average latency: {avg:.0f}ms (vs ~1000ms on T4 = {1000/avg:.1f}x faster)")


In [ ]:
# Cell 7: Massive Batch Inference (B200 Maximum Throughput)
print("⚡ MASSIVE BATCH INFERENCE - B200 Maximum Throughput")

# Generate 64 instructions
batch_insts = [f"Click on Metric {i+1}" for i in range(8)] + \
              [f"Click on {item}" for item in ["Dashboard", "Analytics", "Reports", "Users", 
                                                 "Settings", "Admin", "Logs", "Export All",
                                                 "Generate Report", "Sync Data", "Configure"]] * 5

batch_inputs = [{"prompt": build_prompt(i), "multi_modal_data": {"image": test_image}} 
                for i in batch_insts[:64]]

print(f"\nProcessing {len(batch_inputs)} requests in parallel...")
batch_start = time.time()
batch_outputs = llm.generate(batch_inputs, sampling_params=sampling_params)
batch_time = time.time() - batch_start

print(f"\n✅ Completed in {batch_time:.2f}s")
print(f"📈 Throughput: {len(batch_inputs) / batch_time:.1f} requests/second")
print(f"📈 Per-request: {batch_time / len(batch_inputs) * 1000:.0f}ms")


In [ ]:
# Cell 8: GPU Architecture Comparison Summary
print("""
╔═════════════════════════════════════════════════════════════════════════════════╗
║                        GPU ARCHITECTURE COMPARISON                               ║
╠═════════════════════════════════════════════════════════════════════════════════╣
║                                                                                 ║
║  Metric              │  T4 (Turing)  │  H100 (Hopper)  │  B200 (Blackwell)     ║
║  ────────────────────┼───────────────┼─────────────────┼───────────────────    ║
║  VRAM                │  16 GB        │  80 GB          │  192 GB               ║
║  Bandwidth           │  320 GB/s     │  3,350 GB/s     │  8,000 GB/s           ║
║  FP16 TFLOPS         │  65           │  1,979          │  ~4,000               ║
║  FP8 TFLOPS          │  ❌           │  3,958          │  ~8,000               ║
║  FP4 TFLOPS          │  ❌           │  ❌             │  ~16,000              ║
║  ────────────────────┼───────────────┼─────────────────┼───────────────────    ║
║  Expected Latency    │  ~1000ms      │  ~200ms         │  ~100ms               ║
║  Expected Throughput │  ~1 req/s     │  ~6 req/s       │  ~15 req/s            ║
║  Max Concurrent      │  4            │  64             │  128                  ║
║  Max Context         │  2K           │  32K            │  128K                 ║
║  Max Resolution      │  720×720      │  1920×1080      │  3840×2160 (4K)       ║
║                                                                                 ║
║  UNIQUE B200 FEATURES:                                                          ║
║  ✅ FP4 Tensor Cores (4x throughput when available)                             ║
║  ✅ 2nd Gen Transformer Engine                                                   ║
║  ✅ HBM3e (25x bandwidth vs T4)                                                  ║
║  ✅ NVLink 5.0 (1.8 TB/s multi-GPU)                                              ║
║  ✅ Decompression Engine                                                         ║
║                                                                                 ║
╚═════════════════════════════════════════════════════════════════════════════════╝
""")
print("⚡ B200 NOTEBOOK COMPLETE!")
